In [1]:
# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import deepSI
import ipywidgets as widgets
from scipy.io import loadmat # to load .mat data
from torch import nn
import torch

%matplotlib widget


In [2]:
# load data
# load external data
dataTrainMat = loadmat('VdP_Autonomous_noisy_20Hz.mat')['dataTrain']
dataValMat = loadmat('VdP_Autonomous_noisy_20Hz.mat')['dataVal']
dataTestMat = loadmat('VdP_Autonomous_noisy_20Hz.mat')['dataTest'] # noiseless
dataTestNoiselessMat = loadmat('VdP_Autonomous_noiseless_20Hz.mat')['dataTest'] # noiseless

nStates = 2 # number of system states
nReal = 50 # number of realizations

# structure: dataTrainMat[realizationIndex][0][stateIndex][timeIndex]



In [3]:
# some variables
nf = 50 # length of truncated simulation
nx = 100 # number of states of model
nb = 0 # input history for encoder
na = 1 # output history for encoder
nne = 100 # number of neurons per layer for encoder
nng = 40 # number of neurons per layer for affine input matrix function - if not linear
nnf = 20 # number of neuros state function
nnh = 20 # number of neurons per layer for output function - if not linear
nle = 1 # number of layers encoder
nlg = 1 # number of layers state transition function
nlf = 1
nlh = 1 # number of layers output function

trainEpochs = 10000

In [4]:
# create training validation test data structures

# create deepSI data structure
# trainData = deepSI.System_data(u=uTrain,y=yTrain)
# valData = deepSI.System_data(u=uVal,y=yVal)
# testData = deepSI.System_data(u=uTest,y=yTest)

trainDataList = []
for i in range(0,nReal): # for i=1:nReal
    yTrain = dataTrainMat[i][0][:][0:]
    trainDataTemp = deepSI.System_data(u=None,y=np.transpose(yTrain)) 
    trainDataList.append(trainDataTemp)
    
for i in range(0,nReal-20): # for i=1:nReal 
    yTrain = dataValMat[i][0][:][0:]
    trainDataTemp = deepSI.System_data(u=None,y=np.transpose(yTrain)) 
    trainDataList.append(trainDataTemp)
    
# for i in range(0,nReal-10): # for i=1:nReal 
#     yTrain = dataTestMat[i][0][:][0:]
#     trainDataTemp = deepSI.System_data(u=None,y=np.transpose(yTrain)) 
#     trainDataList.append(trainDataTemp)
    
trainData = deepSI.System_data_list(trainDataList)

valDataList = []
for i in range(nReal-20,nReal): # for i=1:nReal
    yVal = dataValMat[i][0][:][0:]
    valDataTemp = deepSI.System_data(u=None,y=np.transpose(yVal)) 
    valDataList.append(valDataTemp)

valData = deepSI.System_data_list(valDataList)

testDataList = []
for i in range(nReal-10,nReal): # for i=1:nReal
    yTest = dataTestNoiselessMat[i][0][:][0:]
    testDataTemp = deepSI.System_data(u=None,y=np.transpose(yTest)) 
    testDataList.append(testDataTemp)

testData = deepSI.System_data_list(testDataList)


In [14]:
# plot data
# plt.figure()
# plt.plot(trainData.u)
# plt.ylabel('u'); plt.xlabel('time'); plt.legend(['train input'])
# plt.show()

plt.figure()
plt.plot(trainData.y)
plt.ylabel('y'); plt.xlabel('time'); plt.legend(['train output'])
plt.show()

plt.figure()
for t in trainData:
    plt.plot(t.y[:,0],t.y[:,1],',')
plt.show()
                                        
# plt.figure()
# plt.plot(valData.u)
# plt.ylabel('u'); plt.xlabel('time'); plt.legend(['validation input'])
# plt.show()

# plt.figure()
# plt.plot(valData.y)
# plt.ylabel('y'); plt.xlabel('time'); plt.legend(['validation output'])
# plt.show()

# plt.figure()
# plt.plot(testArrowData.u)
# plt.ylabel('u'); plt.xlabel('time'); plt.legend(['test input'])
# plt.show()

# plt.figure()
# plt.plot(testData.y)
# plt.ylabel('y'); plt.xlabel('time'); plt.legend(['test output'])
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# init model structure
from LinearStateOutputFunction import linear_state_net
from LinearStateOutputFunction import linear_output_net
mod_KoopmanEncoder = deepSI.fit_systems.SS_encoder_general(nx=nx, na=na, nb=nb, f_net=linear_state_net, h_net=linear_output_net,
                                    e_net_kwargs={'n_nodes_per_layer':nne, 'n_hidden_layers':nle, 'activation':nn.Tanh}, 
                                    f_net_kwargs={'bias':False}, 
                                    h_net_kwargs={'bias':True})


# mod_KoopmanEncoder = deepSI.fit_systems.SS_encoder_general(nx=nx, na=na, nb=nb,
#                                     e_net_kwargs={'n_nodes_per_layer':nne, 'n_hidden_layers':nle, 'activation':nn.Tanh}, 
#                                     f_net_kwargs={'n_nodes_per_layer':nnf, 'n_hidden_layers':nlf, 'activation':nn.Tanh}, 
#                                     h_net_kwargs={'n_nodes_per_layer':nnh, 'n_hidden_layers':nlh, 'activation':nn.Tanh})

In [43]:

# estimate model using SS-Encoder method
nf = 150
lr = 0.0001
beta1 = 0.7
beta2 = 0.9
trainEpochs = 30000


# mod_KoopmanEncoder.bestfit = float('inf') # reset best validation result


# mod_KoopmanEncoder.checkpoint_load_system('_last') #load last checkpoint
# mod_KoopmanEncoder.optimizer.param_groups[0]['lr'] = lr


mod_KoopmanEncoder.fit(trainData, epochs=trainEpochs, batch_size=256, loss_kwargs={'nf':nf,'dilation':1}, optimizer_kwargs={'lr':lr,'betas':(beta1, beta2)}, sim_val=valData, concurrent_val=True)
# mod_KoopmanEncoder.fit(trainData, epochs=trainEpochs, batch_size=1024, loss_kwargs={'nf':nf,'dilation':1}, optimizer_kwargs=LBFGS_optimizer_kwargs, sim_val=valData, concurrent_val=True)
# mod_KoopmanEncoder.checkpoint_load_system('_last') #load last checkpoint 

# gradually increase the truncated simulation length
# lr = 0.0001
# beta1 = 0.8
# beta2 = 0.9
# trainEpochs = 50
# for nf in range(55,405,5):
#     print('Fitting horizon: ' + str(nf))
#     mod_KoopmanEncoder.fit(trainData, epochs=trainEpochs, batch_size=1024, loss_kwargs={'nf':nf,'dilation':1}, optimizer_kwargs={'lr':lr,'betas':(beta1, beta2)}, sim_val=valData, concurrent_val=True)
#     mod_KoopmanEncoder.checkpoint_load_system('_last') #load last checkpoint 



N_training_samples = 28080, batch_size = 256, N_batch_updates_per_epoch = 109


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch    1, Train sqrt loss  0.1696, Val NRMS 0.2763, Time Loss: 33.1%, back: 39.5%,  4.0 vals/epoch, 5.86 batches/sec
Epoch    2, Train sqrt loss  0.1696, Val NRMS 0.2824, Time Loss: 32.2%, back: 38.3%,  4.5 vals/epoch,  5.8 batches/sec
Epoch    3, Train sqrt loss  0.1695, Val NRMS 0.2777, Time Loss: 31.4%, back: 37.2%, 4.67 vals/epoch, 5.88 batches/sec
Epoch    4, Train sqrt loss  0.1696, Val NRMS 0.2851, Time Loss: 31.4%, back: 36.6%, 4.75 vals/epoch, 5.35 batches/sec
Epoch    5, Train sqrt loss  0.1696, Val NRMS 0.2873, Time Loss: 31.8%, back: 36.7%,  5.0 vals/epoch, 4.85 batches/sec
Epoch    6, Train sqrt loss  0.1695, Val NRMS 0.2805, Time Loss: 31.4%, back: 36.4%, 5.17 vals/epoch, 4.69 batches/sec
Epoch    7, Train sqrt loss  0.1696, Val NRMS 0.2818, Time Loss: 31.7%, back: 36.6%, 5.14 vals/epoch, 4.69 batches/sec
Epoch    8, Train sqrt loss  0.1695, Val NRMS 0.2859, Time Loss: 31.7%, back: 36.8%, 5.12 vals/epoch, 4.63 batches/sec
Epoch    9, Train sqrt loss  0.1696, Val NRMS 0.

In [19]:
mod_KoopmanEncoder.checkpoint_load_system('_last')
mod_KoopmanEncoder.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.7, 0.9)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)

In [38]:
mod_KoopmanEncoder.checkpoint_load_system(name='_last')

# plot validation error evolution
plt.figure(facecolor='w', edgecolor='k')
plt.semilogy(np.sqrt(mod_KoopmanEncoder.Loss_train[:-1]))
plt.semilogy(mod_KoopmanEncoder.Loss_val[:-1])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
mod_KoopmanEncoder.checkpoint_load_system(name='_best')

# validate &plot results
# plot validation error evolution
# validate & plot results
test_simulation_encoder = mod_KoopmanEncoder.apply_experiment(testData)
val_simulation_encoder = mod_KoopmanEncoder.apply_experiment(valData)
train_simulation_encoder = mod_KoopmanEncoder.apply_experiment(trainData)

# display
print('  Encoder-Koopman Affine  Model')
print('NRMS Error')
print('Multisine Train: ' + str(train_simulation_encoder.NRMS(trainData)))
print('Multisine Val: ' + str(val_simulation_encoder.NRMS(valData)))
print('Multisine Test: ' + str(test_simulation_encoder.NRMS(testData))) 
print(' ')
print('RMS Error')
print('Multisine Train: ' + str(train_simulation_encoder.RMS(trainData)))
print('Multisine Val: ' + str(val_simulation_encoder.RMS(valData)))
print('Multisine Test: ' + str(test_simulation_encoder.RMS(testData))) 

plt.figure(facecolor='w', edgecolor='k')
plt.plot(valData.y)
plt.plot(val_simulation_encoder.y,'--')
plt.plot(valData.y-val_simulation_encoder.y,'.')
plt.ylabel('y'); plt.xlabel('x'); plt.legend(['test output', 'simulation error test encoder'])
plt.show()

plt.figure(facecolor='w', edgecolor='k')
plt.plot(testData.y)
plt.plot(test_simulation_encoder.y,'--')
plt.plot(testData.y-test_simulation_encoder.y,'.')
plt.ylabel('y'); plt.xlabel('x'); plt.legend(['test output', 'simulation error test encoder'])
plt.show()

plt.figure(facecolor='w', edgecolor='k')
for v in valData:
    plt.plot(v.y[:,0],v.y[:,1],'b-')
    
for vi in val_simulation_encoder:
    plt.plot(vi.y[:,0],vi.y[:,1],'r--')
    
# for v,vi in zip(valData,val_simulation_encoder):
#     plt.plot(v.y[:,0],v.y[:,1],'b-')
#     plt.plot(vi.y[:,0],vi.y[:,1],'r--')
plt.show()


plt.figure(facecolor='w', edgecolor='k')
for v in testData:
    plt.plot(v.y[:,0],v.y[:,1],'b-')
    
for vi in test_simulation_encoder:
    plt.plot(vi.y[:,0],vi.y[:,1],'r--')
    
# for v,vi in zip(valData,val_simulation_encoder):
#     plt.plot(v.y[:,0],v.y[:,1],'b-')
#     plt.plot(vi.y[:,0],vi.y[:,1],'r--')
plt.show()

  Encoder-Koopman Affine  Model
NRMS Error
Multisine Train: 0.22814372586351422
Multisine Val: 0.2713464520308761
Multisine Test: 0.123497385110894
 
RMS Error
Multisine Train: 0.32805866570772746
Multisine Val: 0.3854975485797013
Multisine Test: 0.18091435957700824


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
cycleData = deepSI.System_data(u=None,y=np.ones((1000,2))) 

limitCycle_simulation_encoder = mod_KoopmanEncoder.apply_experiment(cycleData)

plt.figure(facecolor='w', edgecolor='k')
plt.plot(limitCycle_simulation_encoder.y[:,0],limitCycle_simulation_encoder.y[:,1],',')
plt.show()


plt.figure(facecolor='w', edgecolor='k')
plt.plot(limitCycle_simulation_encoder.y)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
# plot n-step error

plt.figure(facecolor='w', edgecolor='k')
for k in range(0,5):
    plt.plot(mod_KoopmanEncoder.n_step_error(testData[k],nf=400))
    
plt.ylim(0,None)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
## save model
mod_KoopmanEncoder.save_system('VdP_Koopman_Autonomous_100States_20Hz_nf150_noisy_v1.sav')

## load model
# mod_KoopmanEncoder = deepSI.systems.load_system('VdP_Koopman_Autonomous_100States_v2.sav')